In [ ]:
from tabula import read_pdf
from tabulate import tabulate
# import camelot
import pandas as pd
import numpy as np


# pages=','.join(str(i) for i in range(3, 99))

# Using Tabula for Table Extraction

In [ ]:
df = read_pdf('ESG-Frameworks/Mapping-Standars/sdg-gri.pdf', stream=True, pages = 3, area = [80.51, 90.42, 561.96, 814.18], multiple_tables=False )


In [ ]:
for i in range(4, 73):
    df[0] = df[0].append(read_pdf('ESG-Frameworks/Mapping-Standars/sdg-gri.pdf', stream=True, pages = i, area = [80.51, 90.42, 561.96, 814.18], multiple_tables=False )[0], ignore_index=True)
    

In [ ]:
df[0].rename(columns = {'Sources':'Source'}, inplace = True)

In [ ]:
for i in range(74, 99):
    df[0] = df[0].append(read_pdf('ESG-Frameworks/Mapping-Standars/sdg-gri.pdf', stream=True, pages = i, area = [80.51, 90.42, 561.96, 814.18], multiple_tables=False )[0], ignore_index=True)
    

In [ ]:
df[0].to_csv('SDG-GRI-DF.csv')

In [ ]:
# print(tabulate(df[0]))

# Finding links between Frameworks SDG AND GRI

In [ ]:
df = pd.read_csv('SDG-GRI-DF.csv')
# df.head(3)

In [ ]:
df = df.drop(['Unnamed: 0'], axis=1)

In [ ]:
df = df.replace(np.nan, '', regex=True)
df = df.replace('', np.nan).ffill().bfill()


In [ ]:
# df.head(60)

In [ ]:
df = df.groupby(['Target', 'Disclosure'])['Available Business Disclosures'].apply('\n'.join).reset_index()


In [ ]:
# df_part2 = df.groupby(['Target', 'Disclosure'])['Source'].apply(', \n'.join).reset_index()

In [ ]:
df = df[["Target", "Available Business Disclosures", "Disclosure"]]

In [ ]:
df.to_csv('SDG_GRI.csv')

In [ ]:
df_part2

In [ ]:
df.head(60)

In [ ]:
df_part1.to_csv('check.csv')

In [ ]:
df.head(60)

In [ ]:
df.head()

In [ ]:
df

In [ ]:
df